In [1]:
import os
import re
import json
import pandas as pd
import numpy as np
import datetime
import itertools
import warnings
from zoning.utils import get_project_root
from zoning.term_extraction.types import RelevantContext

from typing import List, Tuple, Union, Optional

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


sk


# Not Found exploration

In [24]:
# Set the directory path where the CSV files are located
directory = os.path.join(get_project_root(), 'data', 'logs', 'included_context_phrases', 'not-found')

# List all files in the directory
os.listdir(directory)

['timestamp=2023-11-14_19-40_town=branford-short-beach_district=Business-C_term=min_lot_size_phrase=not-found_tokens=2000_occurrence=not-found.csv',
 'timestamp=2023-11-14_19-40_town=beacon-falls_district=Planned-Adaptive-Reuse-Development-3_term=min_lot_size_phrase=not-found_tokens=2000_occurrence=not-found.csv',
 'timestamp=2023-11-14_19-40_town=bloomfield_district=I-1-General-Industry_term=min_lot_size_phrase=not-found_tokens=2000_occurrence=not-found.csv',
 'timestamp=2023-11-14_19-40_town=branford_district=Residence-R-3_term=min_lot_size_phrase=UNIT-LOT-AREA-PERDWELLING-UNIT10,000-squarefeet_tokens=2000_occurrence=not-found.csv',
 'timestamp=2023-11-14_19-40_town=barkhamsted_district=Restricted-Business_term=min_lot_size_phrase=not-found_tokens=2000_occurrence=not-found.csv',
 'timestamp=2023-11-14_19-40_town=ashford_district=Technology-Development_term=min_lot_size_phrase=not-found_tokens=2000_occurrence=not-found.csv',
 'timestamp=2023-11-14_19-40_town=bridgewater_district=Comme

In [25]:
# Create an empty list to store the dataframes
dfs = []

# Loop through all CSV files in the directory and append them to the list
for filename in os.listdir(directory):
    if filename.endswith('.csv') and filename.startswith('timestamp'):
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        dfs.append(df)

# Concatenate all dataframes in the list into one
df = pd.concat(dfs, ignore_index=True)


# rename the columns to lowercase and replace spaces by underscores
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [26]:
joined_towns = '-'.join(list(set(df.town.to_list())))
joined_terms = '-'.join(list(set(df.term.to_list())))


In [27]:
# Access the appropriate columns for the filename
earliest_timestamp = df.timestamp.min()
latest_timestamp = df.timestamp.max()


earliest_timestamp_dt = datetime.datetime.strptime(earliest_timestamp, '%Y-%m-%d_%H-%M')
latest_timestamp_dt = datetime.datetime.strptime(latest_timestamp, '%Y-%m-%d_%H-%M')

# Calculate the difference
delta = latest_timestamp_dt - earliest_timestamp_dt

# Extract days, hours, and minutes
days = delta.days
hours, remainder = divmod(delta.seconds, 3600)
minutes, _ = divmod(remainder, 60)

# save the joined dataframe as a CSV file in the data/logs/included_context_phrases directory
filename = f"joined_towns={joined_towns[:50]}_term={joined_terms}_first-date={earliest_timestamp}_last-date={latest_timestamp}_delta={days}days-{hours}hours-{minutes}minutes.csv"
df.to_csv(os.path.join(get_project_root(), 'data', 'logs', 'included_context_phrases', 'not-found', filename), index=False)

# checkpoint save
we reload df from checkpoint

In [28]:
# Filter those that start with "joined" and we pick the latest one.
files = os.listdir(directory)
joined_files = [file for file in files if file.startswith('joined')]
latest_file = sorted(joined_files, key=lambda x: os.path.getmtime(os.path.join(directory, x)), reverse=True)[0]

df = pd.read_csv(os.path.join(directory, latest_file))

In [29]:
df

,phrase,town,district.full_name,district.short_name,term,n_tokens,occurrence,before-context,after-context,document,timestamp
0,not-found,branford-short-beach,Business-C,SB-C,min_lot_size,2000,not-found,"\nNEW PAGE 16\nCELL (1, 1): \n6.2.17 Laborator...","t, whichever is less.\nd. In addition to the r...","\nNEW PAGE 16\nCELL (1, 1): \n6.2.17 Laborator...",2023-11-14_19-40
1,not-found,branford-short-beach,Business-C,SB-C,min_lot_size,2000,not-found,\nNEW PAGE 10\n4.7\nNonconformity - Improvemen...,and\n4.8.3\nIf the lot does not meet the area ...,\nNEW PAGE 10\n4.7\nNonconformity - Improvemen...,2023-11-14_19-40
2,not-found,branford-short-beach,Business-C,SB-C,min_lot_size,2000,not-found,\nNEW PAGE 28\nA such sign shall not exceed 20...,"h the Connecticut\nCoastal Management Act, as ...",\nNEW PAGE 28\nA such sign shall not exceed 20...,2023-11-14_19-40
3,not-found,branford-short-beach,Business-C,SB-C,min_lot_size,2000,not-found,\nNEW PAGE 28\nA such sign shall not exceed 20...,"h the Connecticut\nCoastal Management Act, as ...",\nNEW PAGE 28\nA such sign shall not exceed 20...,2023-11-14_19-40
4,not-found,branford-short-beach,Business-C,SB-C,min_lot_size,2000,not-found,\nNEW PAGE 28\nA such sign shall not exceed 20...,"h the Connecticut\nCoastal Management Act, as ...",\nNEW PAGE 28\nA such sign shall not exceed 20...,2023-11-14_19-40
5,not-found,branford-short-beach,Business-C,SB-C,min_lot_size,2000,not-found,\nNEW PAGE 28\nA such sign shall not exceed 20...,"h the Connecticut\nCoastal Management Act, as ...",\nNEW PAGE 28\nA such sign shall not exceed 20...,2023-11-14_19-40
6,not-found,branford-short-beach,Business-C,SB-C,min_lot_size,2000,not-found,\nNEW PAGE 25\n6 ft.\n6. Maximum Building Lot ...,"ver made, displayed, painted, supported or att...",\nNEW PAGE 25\n6 ft.\n6. Maximum Building Lot ...,2023-11-14_19-40
7,not-found,branford-short-beach,Business-C,SB-C,min_lot_size,2000,not-found,\nNEW PAGE 28\nA such sign shall not exceed 20...,"h the Connecticut\nCoastal Management Act, as ...",\nNEW PAGE 28\nA such sign shall not exceed 20...,2023-11-14_19-40
8,not-found,branford-short-beach,Business-C,SB-C,min_lot_size,2000,not-found,\nNEW PAGE 24\n30 ft.\n30 ft.\na. Accessory Bu...,paces required\nherein for residential or comm...,\nNEW PAGE 24\n30 ft.\n30 ft.\na. Accessory Bu...,2023-11-14_19-40
9,not-found,branford-short-beach,Business-C,SB-C,min_lot_size,2000,not-found,\nNEW PAGE 28\nA such sign shall not exceed 20...,"h the Connecticut\nCoastal Management Act, as ...",\nNEW PAGE 28\nA such sign shall not exceed 20...,2023-11-14_19-40


In [30]:
print(df.document[0])


NEW PAGE 16
CELL (1, 1): 
6.2.17 Laboratories, including experimental, research and manufacturing.
CELL (1, 2): 
6.2.17 Laboratories, including experimental, research and manufacturing.
CELL (2, 1): 
6.2.18 Carpentry and woodworking shops.
CELL (2, 2): 
6.2.18 Carpentry and woodworking shops.
CELL (3, 1): 
6.2.19 Stone yards or monument works.
CELL (3, 2): 
6.2.19 Stone yards or monument works.
CELL (4, 1): 
6.2.20 Funeral parlors and undertakers establishments.
CELL (4, 2): 
6.2.20 Funeral parlors and undertakers establishments.
CELL (5, 1): 
6.2.21 Veterinary hospitals, commercial kennels, grooming facilities and animal clinics.
CELL (5, 2): 
6.2.21 Veterinary hospitals, commercial kennels, grooming facilities and animal clinics.
CELL (6, 1): 
6.3
buildings and other structures shall be used for one or more of the following uses:
CELL (6, 2): 
Permitted Uses - Residence Districts A and B: In Residence Districts A and B, land,
buildings and other structures shall be used for one or m

### Add relevant pages that appear in the context

In [31]:
if 'pages' not in df.columns:
    # split document by NEW PAGE
    page_list = df['document'].str.split('\nNEW PAGE').tolist()
    clean_page_list = [[page.split('\n')[0] for page in pages if page.split('\n')[0].strip() != ''] for pages in page_list]
    #print(clean_page_list)

    df['pages'] = pd.Series(clean_page_list)

### Double check (are pages included) 
answer should be FALSE

In [32]:
if 'phrase_included' not in df.columns:
    # split the phrases into a list by \n and separate it into new columns
    df = df.join(df['phrase'].str.split('\n', expand=True).add_prefix('phrase_'))

    # we check if phrase is included (expected not to be)
    df['phrase_included'] = df.apply(lambda x: x['phrase'] in x['document'], axis=1) 

df = df.sort_values(by=['town', 'phrase'], ascending=[True, True])
df

,phrase,town,district.full_name,district.short_name,term,n_tokens,occurrence,before-context,after-context,document,timestamp,pages,phrase_0,phrase_1,phrase_2,phrase_3,phrase_included
103,not-found,andover,Andover-Lake,AL,min_lot_size,2000,not-found,\nNEW PAGE 98\n11.3.1\nReductions in dimension...,ation of rainwater;\nf. Support and promote sa...,\nNEW PAGE 98\n11.3.1\nReductions in dimension...,2023-11-14_19-40,"[ 98, 99, 100, 101, 102]",not-found,None,None,None,False
119,not-found,ansonia,AA-Residence,AA,min_lot_size,2000,not-found,"\nNEW PAGE 166\n166\nCELL (1, 1): \nCELL (1, 2...","le B (p.7), Sec 510.2.1.\nCELL (4, 1): \n2. Ma...","\nNEW PAGE 166\n166\nCELL (1, 1): \nCELL (1, 2...",2023-11-14_19-40,"[ 166, 167, 168]",not-found,None,None,None,False
120,not-found,ansonia,AA-Residence,AA,min_lot_size,2000,not-found,\nNEW PAGE 10\nDwelling-two family: A building...,vation\n\nNEW PAGE 12\na. Listed individual in...,\nNEW PAGE 10\nDwelling-two family: A building...,2023-11-14_19-40,"[ 10, 11, 12, 13]",not-found,None,None,None,False
47,not-found,ashford,Technology-Development,TD,min_lot_size,2000,not-found,"\nNEW PAGE 21\nAshford, Connecticut - Zoning R...",mit Uses\nA special permit may be issued for t...,"\nNEW PAGE 21\nAshford, Connecticut - Zoning R...",2023-11-14_19-40,"[ 21, 22, 23, 24, 25]",not-found,None,None,None,False
48,not-found,ashford,Technology-Development,TD,min_lot_size,2000,not-found,"\nNEW PAGE 21\nAshford, Connecticut - Zoning R...",mit Uses\nA special permit may be issued for t...,"\nNEW PAGE 21\nAshford, Connecticut - Zoning R...",2023-11-14_19-40,"[ 21, 22, 23, 24, 25]",not-found,None,None,None,False
49,not-found,ashford,Technology-Development,TD,min_lot_size,2000,not-found,"\nNEW PAGE 29\nAshford, Connecticut - Zoning R...","neral, interior lots shall be avoided. However...","\nNEW PAGE 29\nAshford, Connecticut - Zoning R...",2023-11-14_19-40,"[ 29, 30, 31]",not-found,None,None,None,False
50,not-found,ashford,Technology-Development,TD,min_lot_size,2000,not-found,"\nNEW PAGE 21\nAshford, Connecticut - Zoning R...",mit Uses\nA special permit may be issued for t...,"\nNEW PAGE 21\nAshford, Connecticut - Zoning R...",2023-11-14_19-40,"[ 21, 22, 23, 24, 25]",not-found,None,None,None,False
51,not-found,ashford,Technology-Development,TD,min_lot_size,2000,not-found,"\nNEW PAGE 21\nAshford, Connecticut - Zoning R...",mit Uses\nA special permit may be issued for t...,"\nNEW PAGE 21\nAshford, Connecticut - Zoning R...",2023-11-14_19-40,"[ 21, 22, 23, 24, 25]",not-found,None,None,None,False
52,not-found,ashford,Technology-Development,TD,min_lot_size,2000,not-found,"\nNEW PAGE 9\nAshford, Connecticut - Zoning Re...","ded from time to time, the General\nStatutes s...","\nNEW PAGE 9\nAshford, Connecticut - Zoning Re...",2023-11-14_19-40,"[ 9, 10, 11]",not-found,None,None,None,False
53,not-found,ashford,Technology-Development,TD,min_lot_size,2000,not-found,"\nNEW PAGE 21\nAshford, Connecticut - Zoning R...",mit Uses\nA special permit may be issued for t...,"\nNEW PAGE 21\nAshford, Connecticut - Zoning R...",2023-11-14_19-40,"[ 21, 22, 23, 24, 25]",not-found,None,None,None,False


In [33]:
# Assuming 'sentence' column contains sentences of the document
def find_sentences_with_phrase(document, phrase):
    """Returns all sentences containing the phrase."""
    sentences = document.split('.')
    return [sentence.strip() for sentence in sentences if phrase in sentence]

# create an empty dictionary to store the results
results = {}

# get all columns that start with 'phrase_' followed by a number
phrase_cols = [col for col in df.columns if re.match(r'^phrase_\d+$', col)]

# iterate over the phrase columns
for col in phrase_cols:
    # Create a temporary column to check if the phrase is in the document
    df[col + '_included'] = df.apply(lambda x: x[col] is not None and x[col] in x['document'], axis=1)
    
    # If the phrase is included, get all sentences where it appears
    df[col + '_sentences'] = df.apply(lambda x: find_sentences_with_phrase(x['document'], x[col]) if x[col + '_included'] else [], axis=1)
    
    # If the phrase is included in any of the rows, add it to the results dictionary
    if df[col + '_included'].any():
        sentences = df.loc[df[col + '_included'], col + '_sentences'].explode().dropna().tolist()
        phrase = df[col].dropna().iloc[0]  # Assuming that the same phrase is repeated in the column
        results[phrase] = sentences

    # Drop temporary columns
    df.drop([col + '_sentences'], axis=1, inplace=True)

# print the results dictionary
#print(results)
with open('not-found-context-results.json', 'w') as f:
    json.dump(results, f, indent=4)

df

,phrase,town,district.full_name,district.short_name,term,n_tokens,occurrence,before-context,after-context,document,timestamp,pages,phrase_0,phrase_1,phrase_2,phrase_3,phrase_included,phrase_0_included,phrase_1_included,phrase_2_included,phrase_3_included
103,not-found,andover,Andover-Lake,AL,min_lot_size,2000,not-found,\nNEW PAGE 98\n11.3.1\nReductions in dimension...,ation of rainwater;\nf. Support and promote sa...,\nNEW PAGE 98\n11.3.1\nReductions in dimension...,2023-11-14_19-40,"[ 98, 99, 100, 101, 102]",not-found,None,None,None,False,False,False,False,False
119,not-found,ansonia,AA-Residence,AA,min_lot_size,2000,not-found,"\nNEW PAGE 166\n166\nCELL (1, 1): \nCELL (1, 2...","le B (p.7), Sec 510.2.1.\nCELL (4, 1): \n2. Ma...","\nNEW PAGE 166\n166\nCELL (1, 1): \nCELL (1, 2...",2023-11-14_19-40,"[ 166, 167, 168]",not-found,None,None,None,False,False,False,False,False
120,not-found,ansonia,AA-Residence,AA,min_lot_size,2000,not-found,\nNEW PAGE 10\nDwelling-two family: A building...,vation\n\nNEW PAGE 12\na. Listed individual in...,\nNEW PAGE 10\nDwelling-two family: A building...,2023-11-14_19-40,"[ 10, 11, 12, 13]",not-found,None,None,None,False,False,False,False,False
47,not-found,ashford,Technology-Development,TD,min_lot_size,2000,not-found,"\nNEW PAGE 21\nAshford, Connecticut - Zoning R...",mit Uses\nA special permit may be issued for t...,"\nNEW PAGE 21\nAshford, Connecticut - Zoning R...",2023-11-14_19-40,"[ 21, 22, 23, 24, 25]",not-found,None,None,None,False,False,False,False,False
48,not-found,ashford,Technology-Development,TD,min_lot_size,2000,not-found,"\nNEW PAGE 21\nAshford, Connecticut - Zoning R...",mit Uses\nA special permit may be issued for t...,"\nNEW PAGE 21\nAshford, Connecticut - Zoning R...",2023-11-14_19-40,"[ 21, 22, 23, 24, 25]",not-found,None,None,None,False,False,False,False,False
49,not-found,ashford,Technology-Development,TD,min_lot_size,2000,not-found,"\nNEW PAGE 29\nAshford, Connecticut - Zoning R...","neral, interior lots shall be avoided. However...","\nNEW PAGE 29\nAshford, Connecticut - Zoning R...",2023-11-14_19-40,"[ 29, 30, 31]",not-found,None,None,None,False,False,False,False,False
50,not-found,ashford,Technology-Development,TD,min_lot_size,2000,not-found,"\nNEW PAGE 21\nAshford, Connecticut - Zoning R...",mit Uses\nA special permit may be issued for t...,"\nNEW PAGE 21\nAshford, Connecticut - Zoning R...",2023-11-14_19-40,"[ 21, 22, 23, 24, 25]",not-found,None,None,None,False,False,False,False,False
51,not-found,ashford,Technology-Development,TD,min_lot_size,2000,not-found,"\nNEW PAGE 21\nAshford, Connecticut - Zoning R...",mit Uses\nA special permit may be issued for t...,"\nNEW PAGE 21\nAshford, Connecticut - Zoning R...",2023-11-14_19-40,"[ 21, 22, 23, 24, 25]",not-found,None,None,None,False,False,False,False,False
52,not-found,ashford,Technology-Development,TD,min_lot_size,2000,not-found,"\nNEW PAGE 9\nAshford, Connecticut - Zoning Re...","ded from time to time, the General\nStatutes s...","\nNEW PAGE 9\nAshford, Connecticut - Zoning Re...",2023-11-14_19-40,"[ 9, 10, 11]",not-found,None,None,None,False,False,False,False,False
53,not-found,ashford,Technology-Development,TD,min_lot_size,2000,not-found,"\nNEW PAGE 21\nAshford, Connecticut - Zoning R...",mit Uses\nA special permit may be issued for t...,"\nNEW PAGE 21\nAshford, Connecticut - Zoning R...",2023-11-14_19-40,"[ 21, 22, 23, 24, 25]",not-found,None,None,None,False,False,False,False,False


In [34]:
# TODO update so that this works with n phrases
def adjust_context_indices(document: str, start_idx: int, end_idx: int, char_budget: int = 500) -> Tuple[int, int]:
    """
    Adjust the start and end indices for a context based on the character budget.
    
    Parameters:
    - document: The document string.
    - start_idx: Starting index of the context.
    - end_idx: Ending index of the context.
    - char_budget: The total character count allowed for the context (default is 500).

    Returns:
    - Tuple of (adjusted_start_idx, adjusted_end_idx).
    """
    pre_context_budget = 100
    pre_context_end = start_idx
    pre_context_start = max(0, pre_context_end - pre_context_budget, document.rfind('\n', 0, pre_context_end) + 1)

    remaining_budget = char_budget - (pre_context_end - pre_context_start) - (end_idx - start_idx)
    post_context_start = end_idx
    post_context_end = min(len(document), post_context_start + remaining_budget)

    return pre_context_start, post_context_end

def get_multiple_matches(document: str, phrase: str) -> List[int]:
    """Return all starting indices of a phrase within a document."""
    starts = []
    idx = document.find(phrase)
    while idx != -1:
        starts.append(idx)
        idx = document.find(phrase, idx + 1)
    return starts

def select_best_candidate(context_details: List[RelevantContext]) -> RelevantContext:
    # Select the best candidates sorting by minimum distance:
    context_details.sort(key=lambda x: x[1])
    return context_details[0]

def get_relevant_context(document: str, *phrases: str) -> List[RelevantContext]:
    valid_phrases = [phrase for phrase in phrases if phrase]
    if not valid_phrases:
        warnings.warn("No valid phrases were supplied.")
        return ""

    if len(valid_phrases) == 1:
        return fallback_context(document, valid_phrases[0], index=-1)
    
    phrase_pairs = list(itertools.combinations(valid_phrases, 2))
    context_details = []
    lines = document.split('\n')

    for pair in phrase_pairs:
        idx_0_starts, idx_1_starts = get_multiple_matches(document, pair[0]), get_multiple_matches(document, pair[1])

        # Handle multiple matches
        if len(idx_0_starts) > 1:
            warnings.warn(f"More than one match found for '{pair[0]}'. Considering all matches.")
        if len(idx_1_starts) > 1:
            warnings.warn(f"More than one match found for '{pair[1]}'. Considering all matches.")
        
        for i_0 in idx_0_starts:
            for i_1 in idx_1_starts:
                if i_0 < i_1:
                    start, end = adjust_context_indices(document, i_0, document.find('\n', i_1) + 1)
                    context = document[start:end]
                    chars = len(context)
                    distance = i_1 - i_0
                    line_idx_0, line_idx_1 = document[:i_0].count('\n') + 1, document[:i_1].count('\n') + 1
                    context_details.append((chars, distance, line_idx_0, line_idx_1, context))
                else:
                    context_details.append(fallback_context(document, pair[0]))

    return select_best_candidate(context_details)

def fallback_context(document: str, phrase: str, index: int = None) -> Tuple[int, int, Optional[int], Optional[int], str]:
    """
    Retrieve a context around a given phrase or index if the phrase is not found.
    
    Parameters:
    - document: The document string.
    - phrase: Target phrase.
    - index: Index of the phrase (default is None).

    Returns:
    - Context details.
    """
    if index is None:
        index = document.find(phrase)
    
    if index == -1:
        warnings.warn(f"Phrase {phrase} was not in the supplied document.")
        start, end = adjust_context_indices(document, len(document) // 2, len(document) // 2)
    else:
        start, end = adjust_context_indices(document, index, index + len(phrase))
    
    context = document[start:end]
    chars = len(context)
    distance = np.inf if index == -1 else 0
    line_idx = document.split('\n').index(phrase) + 1 if index != -1 else None
    return (chars, distance, line_idx, -1, context)


In [37]:
def get_context_for_row(row):
    doc = row['document']
    phrase_1 = row['phrase_0']
    phrase_2 = row['phrase_1']
    phrase_3 = row['phrase_2']
    
    result = get_relevant_context(doc, phrase_1, phrase_2, phrase_3)
    return result

# Apply the function to each row and save the full result to a new column 'context_result'
df['context_result'] = df.apply(get_context_for_row, axis=1)

# Now, extract necessary parts from 'context_result' and create additional columns
df['modified_context'] = df['context_result'].apply(lambda x: x[-1])
df['min_distance'] = df['context_result'].apply(lambda x: x[1])
df['line_idx_0'] = df['context_result'].apply(lambda x: x[2])
df['line_idx_1'] = df['context_result'].apply(lambda x: x[3])

df.drop(['context_result'], axis=1, inplace=True)

#save
filename = f"clean_joined_towns={joined_towns[:25]}_term={joined_terms}_first-date={earliest_timestamp}_last-date={latest_timestamp}_delta={days}days-{hours}hours-{minutes}minutes.csv"
df.to_csv(os.path.join(get_project_root(), 'data', 'logs', 'included_context_phrases', 'not-found', filename), index=False)

/var/folders/wv/b_h63k2j0997cxn743g60jn40000gn/T/ipykernel_21264/561756257.py:91: UserWarning: Phrase not-found was not in the supplied document.
  warnings.warn(f"Phrase {phrase} was not in the supplied document.")


In [42]:
df[df.town =="beacon-falls"]

,phrase,town,district.full_name,district.short_name,term,n_tokens,occurrence,before-context,after-context,document,timestamp,pages,phrase_0,phrase_1,phrase_2,phrase_3,phrase_included,phrase_0_included,phrase_1_included,phrase_2_included,phrase_3_included,modified_context,min_distance,line_idx_0,line_idx_1
12,not-found,beacon-falls,Planned-Adaptive-Reuse-Development-3,PARD-3,min_lot_size,2000,not-found,\nNEW PAGE 120\nARITICLE VI TOWNWIDE REQUIREME...,nt of Section 63 of the Zoning Regulations of ...,\nNEW PAGE 120\nARITICLE VI TOWNWIDE REQUIREME...,2023-11-14_19-40,"[ 120, 121, 122, 123]",not-found,None,None,None,False,False,False,False,False,a) The requirement of Section 63 of the Zoning...,inf,NaN,-1
13,not-found,beacon-falls,Planned-Adaptive-Reuse-Development-3,PARD-3,min_lot_size,2000,not-found,"\nNEW PAGE 33\nARTICLE I: GENERAL PROVISIONS,\...","20'\nCELL (12, 4): \n20'\nCELL (12, 5): \n20'\...","\nNEW PAGE 33\nARTICLE I: GENERAL PROVISIONS,\...",2023-11-14_19-40,"[ 33, 34, 35]",not-found,None,None,None,False,False,False,False,False,"20'\nCELL (12, 4): \n20'\nCELL (12, 5): \n20'\...",inf,NaN,-1
14,not-found,beacon-falls,Planned-Adaptive-Reuse-Development-3,PARD-3,min_lot_size,2000,not-found,\nNEW PAGE 120\nARITICLE VI TOWNWIDE REQUIREME...,nt of Section 63 of the Zoning Regulations of ...,\nNEW PAGE 120\nARITICLE VI TOWNWIDE REQUIREME...,2023-11-14_19-40,"[ 120, 121, 122, 123]",not-found,None,None,None,False,False,False,False,False,a) The requirement of Section 63 of the Zoning...,inf,NaN,-1
15,not-found,beacon-falls,Planned-Adaptive-Reuse-Development-3,PARD-3,min_lot_size,2000,not-found,\nNEW PAGE 120\nARITICLE VI TOWNWIDE REQUIREME...,nt of Section 63 of the Zoning Regulations of ...,\nNEW PAGE 120\nARITICLE VI TOWNWIDE REQUIREME...,2023-11-14_19-40,"[ 120, 121, 122, 123]",not-found,None,None,None,False,False,False,False,False,a) The requirement of Section 63 of the Zoning...,inf,NaN,-1
16,not-found,beacon-falls,Planned-Adaptive-Reuse-Development-3,PARD-3,min_lot_size,2000,not-found,\nNEW PAGE 120\nARITICLE VI TOWNWIDE REQUIREME...,nt of Section 63 of the Zoning Regulations of ...,\nNEW PAGE 120\nARITICLE VI TOWNWIDE REQUIREME...,2023-11-14_19-40,"[ 120, 121, 122, 123]",not-found,None,None,None,False,False,False,False,False,a) The requirement of Section 63 of the Zoning...,inf,NaN,-1
17,not-found,beacon-falls,Planned-Adaptive-Reuse-Development-3,PARD-3,min_lot_size,2000,not-found,\nNEW PAGE 108\nARITICLE VI TOWNWIDE REQUIREME...,"ided into\ncondominiums and rental units, cont...",\nNEW PAGE 108\nARITICLE VI TOWNWIDE REQUIREME...,2023-11-14_19-40,"[ 108, 109, 110]",not-found,None,None,None,False,False,False,False,False,The land within PARD #1 shall not be subdivide...,inf,NaN,-1
18,not-found,beacon-falls,Planned-Adaptive-Reuse-Development-3,PARD-3,min_lot_size,2000,not-found,\nNEW PAGE 120\nARITICLE VI TOWNWIDE REQUIREME...,nt of Section 63 of the Zoning Regulations of ...,\nNEW PAGE 120\nARITICLE VI TOWNWIDE REQUIREME...,2023-11-14_19-40,"[ 120, 121, 122, 123]",not-found,None,None,None,False,False,False,False,False,a) The requirement of Section 63 of the Zoning...,inf,NaN,-1
19,not-found,beacon-falls,Planned-Adaptive-Reuse-Development-3,PARD-3,min_lot_size,2000,not-found,\nNEW PAGE 20\nARTICLE I: GENERAL PROVISIONS\n...,ere the Zoning\nEnforcement Officer cannot det...,\nNEW PAGE 20\nARTICLE I: GENERAL PROVISIONS\n...,2023-11-14_19-40,"[ 20, 21, 22]",not-found,None,None,None,False,False,False,False,False,animals must be one hundred and fifty (150) fe...,inf,NaN,-1
20,not-found,beacon-falls,Planned-Adaptive-Reuse-Development-3,PARD-3,min_lot_size,2000,not-found,\nNEW PAGE 120\nARITICLE VI TOWNWIDE REQUIREME...,nt of Section 63 of the Zoning Regulations of ...,\nNEW PAGE 120\nARITICLE VI TOWNWIDE REQUIREME...,2023-11-14_19-40,"[ 120, 121, 122, 123]",not-found,None,None,None,False,False,False,False,False,a) The requirement of Section 63 of the Zoning...,inf,NaN,-1
21,not-found,beacon-falls,Planned-Adaptive-Reuse-Developmen